In [32]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib

# if you are plotting at the rig computer and want to plot the last debugging
# run images, set this to True.
plot_at_rig = True 
processed_is_CDS_subtracted = True # whether to halve the processed_img size
prog = 7 # sizes from 0-smallest to 7-largest

# to help explore possible settings
reshape_factor = 1 # width is divided by this, height is multiplied by this

Using matplotlib backend: Qt5Agg


In [33]:
path = "C:\\Users\\RedShirtImaging\\Desktop\\PhotoZ_Jan2021\\PhotoZ_upgrades\\PhotoZ\\"

if not plot_at_rig:

    path = ".\\data\\"

    
    plot_all = True
    date = "6-29-21"
    path += date + "\\prog" + str(prog) + "\\"
    
# (internal) dimensions of the quadrants
'''
(= 7 - m_program in PhotoZ)
"DM2K_2048x512.cfg", 7  "200 Hz   2048x1024"
"DM2K_2048x50.cfg",  6  "2000 Hz  2048x100"
"DM2K_1024x160.cfg", 5  "1000 Hz  1024x320"
"DM2K_1024x80.cfg",  4  "2000 Hz  1024x160"
"DM2K_1024x80.cfg",  3  "2000 Hz  512x160"
"DM2K_1024x40.cfg",  2  "4000 Hz  512x80"
"DM2K_1024x30.cfg",  1  "5000 Hz  256x60"
"DM2K_1024x20.cfg"   0  "7500 Hz  256x40"
'''
dimensions = {
    7 : {'height': 512,
         'width': 2048 },
    5 : {'height': 50,
         'width': 2048 },
    5 : {'height': 160,
         'width': 1024 },
    4 : {'height': 80,
         'width': 1024 },
    3 : {'height': 80,
         'width': 1024},
    2 : {'height': 40,
         'width': 1024},
    1 : {'height': 30,
         'width': 1024},
    0 : {'height': 20,
         'width': 1024}
}
height = int(dimensions[prog]['height'] * reshape_factor)
width = int(dimensions[prog]['width'] // reshape_factor)
quadrantSize = height * width * 4

In [34]:
def load_image(image_version, quadrantSize, height, delim=' '):
    height *= 2
    final = np.genfromtxt(path + image_version + ".txt", delimiter = delim) # always tab-delim
    raw = np.genfromtxt(path + "raw-" + image_version + ".txt", delimiter = delim)

    raw_img = raw[:quadrantSize,1].reshape(height,-1)
    final_size = quadrantSize
    if processed_is_CDS_subtracted:
        quadrantSize = quadrantSize // 2
    final_img = final[:final_size,1].reshape(height,-1) # CDS-correct image is half the size. Reset rows were removed

    print("Shaping final image to:",final_size, quadrantSize // height)
    print(raw_img.shape, final_img.shape)
    return raw_img, final_img
    
def plot_image(raw_img, final_img, image_version, image_no):
    fig = plt.figure()

    ax1 = fig.add_subplot(2, 1, 1)
    ax2 = fig.add_subplot(2, 1, 2)

    ax1.set_title("Raw, RLI frame " + str(image_no))
    ax2.set_title("CDS subtracted then Deinterleave, RLI frame " + str(image_no))

    ax1.imshow(raw_img, aspect='auto', cmap='jet')
    ax2.imshow(final_img[1:,:], aspect='auto', cmap='jet')

    fig.subplots_adjust(hspace = 0.6)

    plt.show()
    # save to date-specific directory
    plt.savefig(path + 'readout-RLI-' + image_version + ".png")

In [35]:
def remapQuadrants(img):
    # Place second half to the right of the first half
    h,w = img.shape

    q0 = img[:h//4,:]
    q1 = img[h//4:h//2,:]
    q2 = img[h//2:3*h//4,:]
    q3 = img[3*h//4:,:]
    img = np.zeros((h//2, w*2))
    img[:h//4,:w] = q0
    img[:h//4,w:] = q1
    img[h//4:,:w] = q2
    img[h//4:,w:] = q3
    return img

def normalizeQuadrants(img):
    h,w = img.shape
    img[:h//2,:w//2] = (img[:h//2,:w//2] - np.min(img[:h//2,:w//2])) / np.max(img[:h//2,:w//2])
    img[:h//2,w//2:] = (img[:h//2,w//2:] - np.min(img[:h//2,w//2:])) / np.max(img[:h//2,w//2:])
    img[h//2:,:w//2] = (img[h//2:,:w//2] - np.min(img[h//2:,:w//2])) /  np.max(img[h//2:,:w//2])
    img[h//2:,w//2:] = (img[h//2:,w//2:] - np.min(img[h//2:,w//2:])) /  np.max(img[h//2:,w//2:])
    return img

def plot_remapped(raw_img, final_img, image_version, image_no):
    raw_img_2 = remapQuadrants(raw_img)
    final_img_2 = remapQuadrants(final_img)

    fig = plt.figure()

    ax1 = fig.add_subplot(2, 1, 1)
    ax3 = fig.add_subplot(2, 1, 2)


    ax1.set_title("Raw, Remapped, RLI frame " + str(image_no))
    ax3.set_title("Processed, Remapped, RLI frame " + str(image_no))

    ax1.imshow(raw_img_2, aspect='auto', cmap='jet')
    ax3.imshow(final_img_2, aspect='auto', cmap='jet')

    fig.subplots_adjust(hspace = 0.6)

    plt.show()
    # save to date-specific directory
    plt.savefig(path + 'readout-RLI-remapped' + image_version + ".png")


In [36]:
plot_all = False
image_no_selected = 0

for image_no in [0, 150, 350, 450]:
    if plot_all or image_no_selected == image_no:
        image_version = "full-out" + str(image_no)
        raw_img, final_img = load_image(image_version, quadrantSize, height)
        plot_image(raw_img, final_img, image_version, image_no)
        plot_remapped(raw_img, final_img, image_version, image_no)
        print("Displayed frame " + str(image_no))

Shaping final image to: 4194304 2048
(1024, 4096) (1024, 2048)
Displayed frame 0
